In [0]:
import torch
import numpy as np
import random
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from scipy.special import softmax

In [0]:
def is_attack(model, data, t):
  """
  Inputs
  model: the PyTorch NN model
  data: pertubation + original sample
  t: true class label
  Output
  adv_attack: Whether the new sample is an adversarial attack
  """
  adv_attack = False
  model.eval()
  t_out = model(data)
  t_pred = t_out.argmax(dim=1, keepdim=True)
  if t != t_pred:
    adv_attack = True

  return adv_attack

In [0]:
def gen_attack(N, x_orig, t, delta_max, alpha_min, rho_min, G, num_elite, model, fitness_fun, device):
    # Algo 1 from : https://arxiv.org/pdf/1805.11090.pdf
    #Convergence - best fitness difference within episilon for n generations 
    #
    """
        N : size of population
        x_orig: original example (batch, channel, x, y), batch = 1
        t: true label
        delta_max: maxmimum distance 
        alpha_min: min mutation range (~15%)
        rho_min: min mutation probability (~10%)
        G: # of generations
        num_elites: number of top members to keep for the next generation
        model: the attacked model
        fitness_fun: the objective function used to calculate the fitness
        device: hardware the tensors will be stored on
    """
    # initialize population
    dims = list(x_orig.size())
    dims[0] = N
    # population is an (N, 1, 28, 28) in the case of MNIST
    population = torch.empty(dims, device=device).uniform_(-delta_max, delta_max)
    population = torch.clamp(population + x_orig, 0, 1) - x_orig

    #initialize varaibles used in while loop
    count = 1          #Start with an intial population - so count starts as 1
    crit = 1e-5
    adv_attack = last_best = num_i = num_plat = 0

    #Continue until max num. of iterations or get an adversarial example
    while adv_attack != True and count < G:
      if count % 100 == 0:
        print("Generation " + str(count))
      # Find fitness for every individual and save the best fitness
      fitness = fitness_fun(model, population + x_orig, t)
      best_fit = min(fitness)

      #Check to if fitness last two generations is the same, update num_plat
      if abs(best_fit - last_best) <= crit:
        num_i += 1
        if num_i % 100 == 0 and num_i != 0:
          print("Plateau at Generation " + str(count))
          num_plat += 1
      else:
        num_i = 0

      # TODO: This block sorts the population by fitness,
      # can we use this:
      # new_pop = population.clone()[sorted_inds]
      # or
      # new_pop = population.clone()[fitness.argsort()]
      
      # TODO: we can use sorted_inds = fitness.argsort() instead for simplicity
      # Get sorted indices (Ascending!)
      _, sorted_inds = fitness.sort() 
      #Initialize new population by adding the elites
      new_pop = torch.zeros_like(population)
      for i in range(num_elite):
          new_pop[i] = population[sorted_inds[i]]
      
      #The best individual is the one with the best fitness
      best = new_pop[0]
      
      adv_attack = is_attack(model, best + x_orig, t)
      #If not a true adversarial example need to go to next generation
      if adv_attack == False:
        alpha = max(alpha_min, 0.5 * (0.9 ** num_plat))
        rho = max(rho_min, 0.4 * (0.9 ** num_plat))
        # Softmax fitnesses
        soft_fit = F.softmax(-fitness, dim=0) # Negate fitness since we're trying to minimize
        #need to get apply selection and get a new population
        child_pop = selection(population, soft_fit, x_orig, count, alpha, rho, delta_max, num_elite)
        new_pop[num_elite:] = child_pop
        population = new_pop
        count += 1
        ## Need to retain best fitness
        last_best = best_fit

    return best, adv_attack, count

In [0]:
def mutation_op(cur_pop, x_orig, idx, alpha=0.15, rho=0.1, delta_max=0.1):
    """
        cur_pop: the current population
        x_orig :  the image we are using for the attack
        idx    : an index. useful for debugging
        alpha: mutation range
        rho: mutation probability
        delta_max: maxmimum distance
    """
    step_noise = alpha * delta_max
    perturb_noise = torch.empty_like(cur_pop).uniform_(-step_noise, step_noise)
    mask = torch.empty_like(cur_pop).bernoulli_(rho)
    mutated_pop = perturb_noise * mask + cur_pop
    clamped_mutation_pop = torch.clamp(mutated_pop + x_orig, 0, 1) - x_orig
    normalized_pop = torch.clamp(clamped_mutation_pop, -delta_max, delta_max)
    return normalized_pop

# x = torch.FloatTensor(2, 2)
# image = torch.FloatTensor(2, 2).uniform_(0, 1)
# print(x, image)
# mutpop = mutation_op(x, image, 1)

In [0]:
def fitness(model, batch, target_class):
    """
        model: a pytorch neural network model (takes x to log probability)
        batch: a batch of examples
        target_class: the class label of x (as an integer)
        returns: tensor of fitnesses
        This source code is inspired by:
        https://github.com/nesl/adversarial_genattack/blob/2304cdc2a49d2c16b9f43821ad8a29d664f334d1/genattack_tf2.py#L39
    """
    # TODO: does it seem wonky to anyone that we are passing in pop + x_orig
    # into the fitness? this might be a design flaw of ours
    model.eval()
    with torch.no_grad():
        log_probs = model(batch)
        s = - torch.expm1(log_probs[:, target_class]) # Sum of probabilities, minus target class, might be more numerically stabe
        f = log_probs[:, target_class] - torch.log(s)
        return torch.clamp(f.flatten(), -1000, 1000) # clamping to avoid the "all inf" problem

In [0]:
def selection(population, soft_fit, data, idx, alpha, rho, delta_max, num_elite = 2):
    """
        Input
        population: the population of individuals
        soft_fit: the softmax of the fitnesses 
        data: the input value to find a perturbation for
        idx: the generation number we're on - for debugging purposes
        alpha: mutation range
        rho: mutation probability
        num_elite: the number of elites to carry on from the previous generations
        Output
        mut_child_pop: Returns the mutated population of children
    """

    # Crossover
    cdims = list(population.size())
    child_pop_size = population.size()[0] - num_elite
    cdims[0] = child_pop_size
    child_pop = torch.empty(cdims, device=data.device)
    # Roulette
    roulette = Categorical(probs=soft_fit)
    for i in range(child_pop_size):
        parent1_idx = roulette.sample()
        soft_fit_nop1 = soft_fit.clone() + 0.0001 # Incrementing by epsilon to avoid the "all zeros" problem
        soft_fit_nop1[parent1_idx] = 0
        roulette2 = Categorical(probs=soft_fit_nop1)
        parent2_idx = roulette2.sample()
        child = crossover(population[parent1_idx], population[parent2_idx], soft_fit[parent1_idx], soft_fit[parent2_idx])
        child_pop[i] = child
    
    # Mutation
    mut_child_pop = mutation_op(child_pop, data, idx, alpha, rho, delta_max)

    return mut_child_pop


In [0]:
def crossover(parent1, parent2, p1, p2):
    """
    Element-wise crossover
    parent1: individual in old population
    parent2: individual in old population
    p1: softmaxed fitness for parent1
    p2: softmaxed fitness for parent2
    output
    child: new individual from mating of parents
    """
    p = p1/(p1 + p2)
    mask = torch.empty_like(parent1).bernoulli_(p)
    child = mask * parent1 + (1 - mask) * parent2
    return child